<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/Project/langchain_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangSmith : https://smith.langchain.com/

In [3]:
# !pip install langchain-openai
# !pip install langchain
# !pip install -U langchain-community
# !pip install langchainhub
!pip install -qU huggingface_hub
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 25.7 MB/s eta 0:00:00


In [2]:
import os
# from google.colab import userdata
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "donghuna_proj"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANG_SMITH')


In [20]:
system_prompt = """당신은 표(재무제표) 를 해석하는 금융 AI 어시스턴트 입니다.
당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다."""

# 질의내용
question = "대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input}")
])

# 객체 생성
llm = ChatOpenAI(
    model_name="gpt-4o-mini",  # 모델명
    api_key=userdata.get('OPEN-AI_KEY'),
    organization=userdata.get('MY_OPENAI_ORG_ID'),
    streaming=False,
    temperature=0.2  # 창의성 (0.0 ~ 2.0)
).bind(logprobs=True)

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke(question)

# 질의
# answer = llm.stream(question)
# for token in answer:
#     print(token.content, end="", flush=True)
# response = llm.invoke(question)
# print(response.content)
# print(response.response_metadata)



'물론입니다! 대한민국의 아름다운 관광지 10곳과 그 주소를 아래에 정리해 드리겠습니다.\n\n1. **경복궁**\n   - 주소: 서울특별시 종로구 사직로 161\n\n2. **부산 해운대 해수욕장**\n   - 주소: 부산광역시 해운대구 해운대해변로 140\n\n3. **제주도 한라산**\n   - 주소: 제주특별자치도 제주시 한라산로 2070\n\n4. **경주 불국사**\n   - 주소: 경상북도 경주시 불국로 385\n\n5. **전주 한옥마을**\n   - 주소: 전라북도 전주시 완산구 기린대로 99\n\n6. **서울 남산타워 (N서울타워)**\n   - 주소: 서울특별시 용산구 남산공원길 105\n\n7. **속초 설악산 국립공원**\n   - 주소: 강원도 속초시 설악산로 173\n\n8. **인천 송도 센트럴파크**\n   - 주소: 인천광역시 연수구 송도동 1-1\n\n9. **여수 오동도**\n   - 주소: 전라남도 여수시 오동도로 1\n\n10. **강릉 경포대**\n    - 주소: 강원도 강릉시 강릉대로 365\n\n이곳들은 각기 다른 매력을 지닌 관광지로, 자연경관과 역사적 가치가 뛰어난 장소들입니다. 여행 계획에 도움이 되길 바랍니다!'

스트리밍 출력

In [21]:
print(f"ChatOpenAI: {chain.is_lc_serializable()}")

ChatOpenAI: True


In [4]:
from huggingface_hub import login

login(token=userdata.get('HUG_TOKEN'))

NameError: name 'userdata' is not defined

In [4]:
from langchain.prompts import PromptTemplate

template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate.from_template(template)

Huggingface Endpoint

In [8]:
from langchain_huggingface import HuggingFaceEndpoint

# 사용할 모델의 저장소 ID를 설정합니다.
# repo_id = "microsoft/Phi-3-mini-4k-instruct"
repo_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# repo_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,  # 모델 저장소 ID를 지정합니다.
    max_new_tokens=256,  # 생성할 최대 토큰 길이를 설정합니다.
    temperature=0.1,
    huggingfacehub_api_token=userdata.get('HUG_TOKEN'),  # 허깅페이스 토큰
)

# LLMChain을 초기화하고 프롬프트와 언어 모델을 전달합니다.
chain = prompt | llm | StrOutputParser()
# 질문을 전달하여 LLMChain을 실행하고 결과를 출력합니다.
response = chain.invoke({"question": "what is the capital of South Korea?"})
print(response)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


BadRequestError:  (Request ID: 7sHgCHXxr8eiZE7PmRDHW)

Bad request:
Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.

In [17]:
import transformers
import torch

# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    model_kwargs={"torch_dtype": torch.float32},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_id = "donghuna/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Once upon a time, there was a young girl named Sophia who lived in a small village surrounded'}]


In [7]:
device = 0 if torch.cuda.is_available() else -1

# 텍스트 생성 파이프라인 초기화
generator = pipeline('text-generation',
                     model=model,
                     tokenizer=tokenizer,
                     pad_token_id=tokenizer.eos_token_id,
                     device=device)

# 텍스트 생성 예제
text = "Once upon a time,"
generated_texts = generator(text)
print(generated_texts)

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 

In [15]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Hello?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': "Yer, mateyarr, I be ye be need some help! Yer lookin' to have a bit o' a swaboard! Yer question fer a chat? What be ye want to ask me to discuss?"}


모델 로드 및 푸시하기